In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### **INTRO**

- Let's import the required libraries

In [2]:
import pandas as pd
import numpy as np


import plotly 
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import iplot
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

In [3]:
from google.colab import files
uploaded = files.upload()

Saving london_merged.csv to london_merged.csv


### Overview Stage (Initial Data Understanding)

- Read the csv
- Look for basic information about the dataset

In [14]:
df = pd.read_csv('/content/london_merged.csv')
df.head()

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0


Metadata:
- "timestamp" - timestamp field for grouping the data
- "cnt" - the count of a new bike shares
- "t1" - real temperature in C
- "t2" - temperature in C "feels like"
- "hum" - humidity in percentage
- "windspeed" - wind speed in km/h
- "weathercode" - category of the weather
- "isholiday" - boolean field - 1 holiday / 0 non holiday
- "isweekend" - boolean field - 1 if the day is weekend
- "season" - category field meteorological seasons: 0-spring ; 1-summer; 2-fall; 3-winter.

- "weathe_code" category description:
   - 1 = Clear ; mostly clear but have some values with haze/fog/patches of fog/ fog in vicinity 2 = scattered clouds / few clouds 3 = Broken clouds 4 = Cloudy 7 = Rain/ light Rain shower/ Light rain 10 = rain with thunderstorm 26 = snowfall 94 = Freezing Fog

In [15]:
df.shape

(17414, 10)

- We have 17414 instances with 10 different variables to work on.

In [16]:
df.isnull().sum()

timestamp       0
cnt             0
t1              0
t2              0
hum             0
wind_speed      0
weather_code    0
is_holiday      0
is_weekend      0
season          0
dtype: int64

- Yes, very clean data for the 17414 instances.
- In the real world very hard to find this kind of clean data. Enjoy !!

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17414 entries, 0 to 17413
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     17414 non-null  object 
 1   cnt           17414 non-null  int64  
 2   t1            17414 non-null  float64
 3   t2            17414 non-null  float64
 4   hum           17414 non-null  float64
 5   wind_speed    17414 non-null  float64
 6   weather_code  17414 non-null  float64
 7   is_holiday    17414 non-null  float64
 8   is_weekend    17414 non-null  float64
 9   season        17414 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 1.3+ MB


- It looks like we have 9 numeric variable. But is that so???
- Also we have 1 non-numeric variable. 
- Non-numeric variable is coded as Object, but it looks like time object. It needs further adjustment. Noted.
- Also boolean variables are coded as 0 and 1, noted.
- Categorical variables **season** and **weathercode** are also coded as numerical.  Noted.
- "t1" - real temperature in C and "t2" - temperature in C "feels like" seems quite same thing, needs to look their correlation. Noted.

In [18]:
df.drop(['season', 'weather_code', 'is_holiday','is_weekend'], axis=1).describe().T

,count,mean,std,min,25%,50%,75%,max
cnt,17414.0,1143.101642,1085.108068,0.0,257.0,844.0,1671.75,7860.0
t1,17414.0,12.468091,5.571818,-1.5,8.0,12.5,16.00,34.0
t2,17414.0,11.520836,6.615145,-6.0,6.0,12.5,16.00,34.0
hum,17414.0,72.324954,14.313186,20.5,63.0,74.5,83.00,100.0
wind_speed,17414.0,15.913063,7.894570,0.0,10.0,15.0,20.50,56.5


Before going further, let's summarize what we have got from the dataset.

- Our dataset has 17414 time records of the bike rent. 
-  "t1" - real temperature in C and "t2" - temperature in C "feels like" seems quite same thing, needs to look their correlation. We need to be careful about the multicollinearity.

- We have date object, needs to be adjusted.

- Numerically coded (season and weather_code) variables can be used as a group to see the differences among them.

- 'cnt' : count of bike share, will be our target variable to work on it.

- Numerical columns most probably have outliers. (Mean- Median difference, difference between 75% and maximum value, difference between %25 and minimum value), we have to check them.

- Let's make the necessary adjustments before moving to the analysis part.

In [69]:
df[df['t1']==-1.5]

,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season,year_month,year,month,day_of_week,hour,season1
timestamp,,,,,,,,,,,,,,,
2015-01-23 06:00:00,426,-1.5,-2.5,89.5,5.0,1.0,0.0,0.0,3.0,2015-01,2015,1,4,6,Winter


In [68]:
df[df['cnt'] == 0]

,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season,year_month,year,month,day_of_week,hour,season1
timestamp,,,,,,,,,,,,,,,
2015-03-29 01:00:00,0,10.5,8.5,77.0,33.0,7.0,0.0,1.0,0.0,2015-03,2015,3,6,1,Spring


- ternyata ada satu data jumlah bike share yang bernilai 0 (jam 1 malam hari sabtu ngapain ada yg sepedah)

#### **Temperature**

- Lets' check correlation between real temperature and felt temperature.
- if correlation is high, we can detect the multicollinearity and use one of the highly correlated variable  to improve our model success.
- Even though, we will make detailed EDA in this study, still it is best practice to follow.

In [19]:
df['t1'].corr(df['t2'])

0.9883442218765799

- Correlation is extremely high, so we will use only  "t1" - real temperature in C, in our analysis.

In [20]:
df['cnt'].corr(df['t1'])

0.38879845125473067

- Penggunaan Sepedah dengan temperature tidak terlalu dipengaruhi

#### **timestamp** (Convert Data Type)

- Let's make 'timestamp' as datetime object and use its values to make new columns out of it.

In [21]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17414 entries, 0 to 17413
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   timestamp     17414 non-null  datetime64[ns]
 1   cnt           17414 non-null  int64         
 2   t1            17414 non-null  float64       
 3   t2            17414 non-null  float64       
 4   hum           17414 non-null  float64       
 5   wind_speed    17414 non-null  float64       
 6   weather_code  17414 non-null  float64       
 7   is_holiday    17414 non-null  float64       
 8   is_weekend    17414 non-null  float64       
 9   season        17414 non-null  float64       
dtypes: datetime64[ns](1), float64(8), int64(1)
memory usage: 1.3 MB


In [22]:
df= df.set_index('timestamp')

In [23]:
df['year_month'] = df.index.strftime('%Y-%m')
df['year'] = df.index.year
df['month'] = df.index.month
df['day_of_week'] = df.index.dayofweek
df['hour'] = df.index.hour

df.head()

,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season,year_month,year,month,day_of_week,hour
timestamp,,,,,,,,,,,,,,
2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0,2015-01,2015,1,6,0
2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0,2015-01,2015,1,6,1
2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0,2015-01,2015,1,6,2
2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0,2015-01,2015,1,6,3
2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0,2015-01,2015,1,6,4


- Seems much better

#### Look at the **season** and **weather_code** (Code Univariate Analysis)

In [24]:
df['season'].value_counts()

0.0    4394
1.0    4387
3.0    4330
2.0    4303
Name: season, dtype: int64

- category field meteorological seasons: 0-spring ; 1-summer; 2-fall; 3-winter.

- That's good, it can be used as a group to see the differences at the count of bike share

In [25]:
df['weather_code'].value_counts()

1.0     6150
2.0     4034
3.0     3551
7.0     2141
4.0     1464
26.0      60
10.0      14
Name: weather_code, dtype: int64

- "weathe_code" category description:
   - 1 = Clear ; mostly clear but have some values with haze/fog/patches of fog/ fog in vicinity 2 = scattered clouds / few clouds 3 = Broken clouds 4 = Cloudy 7 = Rain/ light Rain shower/ Light rain 10 = rain with thunderstorm 26 = snowfall 94 = Freezing Fog

- It seems OK, can be used in the groupby.

### Analysis Part (Univariate Analysis)

#### **Season**

In [28]:
df['season'].value_counts(normalize=True)

0.0    0.252326
1.0    0.251924
3.0    0.248651
2.0    0.247100
Name: season, dtype: float64

- Dataset contains almost same number of instances from the four seasons.

#### **weather_code**

In [29]:
df['weather_code'].value_counts(normalize=True)

1.0     0.353164
2.0     0.231653
3.0     0.203916
7.0     0.122947
4.0     0.084070
26.0    0.003446
10.0    0.000804
Name: weather_code, dtype: float64

- %35 of the times, weather code is Clear(1.0)
- %23 of the times, weather code is 'scattered clouds, few clouds'
- %20 of the times, weather code is 'broken clouds'
- %12 of the time 'rain, light rain'

- By the way, remember that we are looking at the London's data. So rain and cloud is quite a Londonish.

#### **Count of a New Bike Shares**

In [26]:
df['cnt'].describe()

count    17414.000000
mean      1143.101642
std       1085.108068
min          0.000000
25%        257.000000
50%        844.000000
75%       1671.750000
max       7860.000000
Name: cnt, dtype: float64

- We have huge difference between mean and median values (mean = 1143, median=844)
- It has highly skewed distribution with the outliers on the maximum side.
- We can expect highly right skewed distribution with possible outliers in the maximum side.
- Let' see it.

#### **real temperature in C**

In [27]:
df['t1'].describe()

count    17414.000000
mean        12.468091
std          5.571818
min         -1.500000
25%          8.000000
50%         12.500000
75%         16.000000
max         34.000000
Name: t1, dtype: float64

- Both mean and median scores are very close to each other. Median is slightly higher than mean score. 
- So we can expect very slightly left skewed distribution
- But the distribution will be very close to normal distribution with several outliers.
- Let's see it.

#### **Wind Speed**

In [30]:
df['wind_speed'].describe()

count    17414.000000
mean        15.913063
std          7.894570
min          0.000000
25%         10.000000
50%         15.000000
75%         20.500000
max         56.500000
Name: wind_speed, dtype: float64

- We can expect slighlt right skewed distribution (mean 15.9, median=15)
- Which will be very close to normal distribution
- We can expect outliers on the maximum side.

#### **Humidity**

In [31]:
df['hum'].describe()

count    17414.000000
mean        72.324954
std         14.313186
min         20.500000
25%         63.000000
50%         74.500000
75%         83.000000
max        100.000000
Name: hum, dtype: float64

- Both mean and median scores are close to each other.
- Since median score is little bit higher than mean score, we can expect slightly left skewed distribution.
- Possible outliers on the minimum side.

- As we expected, left skewed distribution with outliers on the left side.

- Ok After seeing numerical variables in detail. let's see correlation matrix and their relationships with count of number of bike share.

### **Correlation**

In [32]:
df[['cnt','t1','hum','wind_speed']].corr()

,cnt,t1,hum,wind_speed
cnt,1.000000,0.388798,-0.462901,0.116295
t1,0.388798,1.000000,-0.447781,0.145471
hum,-0.462901,-0.447781,1.000000,-0.287789
wind_speed,0.116295,0.145471,-0.287789,1.000000


- Based on the correlation matrix:
    - There is a weak positive relationship (.388) between temperature and the number of bike share
    - Also there is a weak negative relationship (.46) between humidity and the number of the bike share.

#### **Holiday or No?**

In [33]:
df['is_holiday'].value_counts()

0.0    17030
1.0      384
Name: is_holiday, dtype: int64

#### **Wekend or No**

In [34]:
df['is_weekend'].value_counts()

0.0    12444
1.0     4970
Name: is_weekend, dtype: int64

- Ok let's go deeper.

# Bivariate Analysis

#### **Bike Share by Seasons**

In [35]:
df['season1']= df['season'].replace({0:'Spring',1:'summer',2:'Fall',3:'Winter'})

In [36]:
df['season1'].value_counts(normalize=True)

Spring    0.252326
summer    0.251924
Winter    0.248651
Fall      0.247100
Name: season1, dtype: float64

- As we have seen in the year_month, same is true based on the seasons.
- Bike share increases on the summer time and reaches lowest point on the winter time.

In [48]:
df.groupby(['season1', 'is_weekend']).agg({'cnt': 'sum', 't1': np.mean})#.reset_index()

cnt         t1
season1 is_weekend                    
Fall    0.0         3889779  13.301592
        1.0         1183261  12.380778
Spring  0.0         3626468  10.660879
        1.0         1223768  10.681275
Winter  0.0         2835113   7.556509
        1.0          722974   8.011272
summer  0.0         4696856  18.362739
        1.0         1727753  18.603448

- Bisa terdapat hipotesis bahwa sepedah digunakan untuk hari kerja dan saat liburan atau weekend itu bersifat rekreasi sehingga tidak sebanyak hari kerja

In [47]:
pd.pivot_table(df, values=['cnt'], index=['season1'], aggfunc={'cnt': 'sum'})

,cnt
season1,
Fall,5073040
Spring,4850236
Winter,3558087
summer,6424609


In [44]:
pd.pivot_table?

#### **Bike Share During the Holiday**

In [61]:
holiday = df.groupby('is_holiday')['cnt'].mean().reset_index().rename(columns={'is_holiday': 'Holiday', 'cnt':'Number of Bike Shared'}, )
holiday['Holiday']= holiday['Holiday'].replace({0: 'Normal Day', 1:'Holiday'})

fig = px.bar(holiday, x='Holiday', y= 'Number of Bike Shared', color='Holiday', )
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

In [62]:
holiday

,Holiday,Number of Bike Shared
0,Normal Day,1151.525191
1,Holiday,769.526042


- Normal days have more bike share than holidays.

In [63]:
holiday = df.groupby('is_holiday').agg({'cnt': 'mean'}).reset_index()
holiday = holiday.rename(columns = {'is_holiday': 'Holiday Status', 'cnt': 'Average Bike Usage'})
holiday['Holiday Status'] = holiday['Holiday Status'].replace({0: 'Normal Day', 1: 'Holiday'})

fig = px.bar(holiday, x='Holiday Status', y= 'Average Bike Usage', color='Holiday Status', )
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

In [64]:
holiday

,Holiday Status,Average Bike Usage
0,Normal Day,1151.525191
1,Holiday,769.526042


#### **Bike Share During the Weekend**

In [65]:
weekend = df.groupby('is_weekend')['cnt'].mean().reset_index().rename(columns={'is_weekend': 'Weekend', 'cnt':'Number of Bike Shared'}, )
weekend['Weekend']= weekend['Weekend'].replace({0: 'Weekday', 1:'Weekend'})

fig = px.bar(weekend, x='Weekend', y= 'Number of Bike Shared', color='Weekend')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()
weekend

,Weekend,Number of Bike Shared
0,Weekday,1209.274831
1,Weekend,977.415694


- Weekdays have more bike share than weekends.